In [ ]:
# Load data and set paths
import pandas as pd
from pathlib import Path
import os

# Auto-detect data path
DATA_PATH = Path("water_dataX.csv")
if not DATA_PATH.exists():
    DATA_PATH = Path("../extract/water_dataX.csv")
if not DATA_PATH.exists():
    DATA_PATH = Path("WQ_combined_clean.csv")
if not DATA_PATH.exists():
    raise FileNotFoundError("Could not find data file. Please set DATA_PATH manually.")

# Load data
try:
    df = pd.read_csv(DATA_PATH, encoding='utf-8', low_memory=False)
except UnicodeDecodeError:
    df = pd.read_csv(DATA_PATH, encoding='latin1', low_memory=False)

# Set model output path
MODEL_OUTPUT = Path("rf_water_model.joblib")
if not MODEL_OUTPUT.parent.exists():
    MODEL_OUTPUT = Path("../extract/rf_water_model.joblib")

print(f"Loaded {len(df)} rows from {DATA_PATH}")
print(f"Columns: {list(df.columns)}")
df.head()



Loaded 1991 rows from water_dataX.csv
Columns: ['STATION CODE', 'LOCATIONS', 'STATE', 'Temp', 'D.O. (mg/l)', 'PH', 'CONDUCTIVITY (µmhos/cm)', 'B.O.D. (mg/l)', 'NITRATENAN N+ NITRITENANN (mg/l)', 'FECAL COLIFORM (MPN/100ml)', 'TOTAL COLIFORM (MPN/100ml)Mean', 'year']


,STATION CODE,LOCATIONS,STATE,Temp,D.O. (mg/l),PH,CONDUCTIVITY (µmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean,year
0,1393,"DAMANGANGA AT D/S OF MADHUBAN, DAMAN",DAMAN & DIU,30.6,6.7,7.5,203,NAN,0.1,11,27,2014
1,1399,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOI...,GOA,29.8,5.7,7.2,189,2,0.2,4953,8391,2014
2,1475,ZUARI AT PANCHAWADI,GOA,29.5,6.3,6.9,179,1.7,0.1,3243,5330,2014
3,3181,RIVER ZUARI AT BORIM BRIDGE,GOA,29.7,5.8,6.9,64,3.8,0.5,5382,8443,2014
4,3182,RIVER ZUARI AT MARCAIM JETTY,GOA,29.5,5.8,7.3,83,1.9,0.4,3428,5500,2014


In [ ]:
# Clean and rename columns to match expected feature names
import numpy as np

# Rename columns to standardized names
rename_map = {
    'Temp': 'Temp', 'Temperature': 'Temp',
    'D.O. (mg/l)': 'DO', 'DO': 'DO', 'Dissolved Oxygen': 'DO',
    'PH': 'pH', 'pH': 'pH',
    'CONDUCTIVITY (µmhos/cm)': 'Conductivity', 'Conductivity': 'Conductivity',
    'B.O.D. (mg/l)': 'BOD', 'BOD': 'BOD',
    'NITRATENAN N+ NITRITENANN (mg/l)': 'Nitrate', 'Nitrate': 'Nitrate',
    'FECAL COLIFORM (MPN/100ml)': 'FecalColiform', 'FecalColiform': 'FecalColiform',
    'TOTAL COLIFORM (MPN/100ml)Mean': 'TotalColiform', 'TotalColiform': 'TotalColiform',
    'STATION CODE': 'StationCode', 'StationCode': 'StationCode',
    'LOCATIONS': 'MonitoringLocation', 'MonitoringLocation': 'MonitoringLocation'
}

df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

# Convert numeric columns
numeric_cols = ['Temp', 'DO', 'pH', 'Conductivity', 'BOD', 'Nitrate', 'FecalColiform', 'TotalColiform']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print(f"Cleaned columns: {[c for c in numeric_cols if c in df.columns]}")
df.head()



Cleaned columns: ['Temp', 'DO', 'pH', 'Conductivity', 'BOD', 'Nitrate', 'FecalColiform', 'TotalColiform']


,StationCode,MonitoringLocation,STATE,Temp,DO,pH,Conductivity,BOD,Nitrate,FecalColiform,TotalColiform,year
0,1393,"DAMANGANGA AT D/S OF MADHUBAN, DAMAN",DAMAN & DIU,30.6,6.7,7.5,203.0,NaN,0.1,11.0,27.0,2014
1,1399,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOI...,GOA,29.8,5.7,7.2,189.0,2.0,0.2,4953.0,8391.0,2014
2,1475,ZUARI AT PANCHAWADI,GOA,29.5,6.3,6.9,179.0,1.7,0.1,3243.0,5330.0,2014
3,3181,RIVER ZUARI AT BORIM BRIDGE,GOA,29.7,5.8,6.9,64.0,3.8,0.5,5382.0,8443.0,2014
4,3182,RIVER ZUARI AT MARCAIM JETTY,GOA,29.5,5.8,7.3,83.0,1.9,0.4,3428.0,5500.0,2014


In [ ]:
# Create Risk column based on WHO/BIS thresholds
THRESHOLDS = {
    'pH': {'low': 6.5, 'high': 8.5},
    'DO': {'high': 3.0, 'medium': 5.0},  # <3 High, <5 Medium
    'BOD': {'medium': 1.0, 'high': 3.0},  # >3 High, >1 Medium
    'Conductivity': {'medium': 500, 'high': 1500},
    'Nitrate': {'medium': 10, 'high': 45},
    'TotalColiform': {'medium': 500, 'high': 2500},
    'FecalColiform': {'medium': 100, 'high': 500}
}

# Score each parameter (0=Low, 1=Medium, 2=High)
for param in ['pH', 'DO', 'BOD', 'Conductivity', 'Nitrate', 'TotalColiform', 'FecalColiform']:
    if param not in df.columns:
        continue
    df[f'{param}_score'] = 0  # Default Low
    col = df[param]
    
    if param == 'pH':
        # pH: outside 6.5-8.5 is High, edge zones are Medium
        df.loc[(col < 6.5) | (col > 8.5), f'{param}_score'] = 2
        df.loc[((col >= 6.5) & (col < 7.0)) | ((col > 8.0) & (col <= 8.5)), f'{param}_score'] = 1
    elif param == 'DO':
        # DO: lower is worse
        df.loc[col < THRESHOLDS[param]['high'], f'{param}_score'] = 2
        df.loc[(col >= THRESHOLDS[param]['high']) & (col < THRESHOLDS[param]['medium']), f'{param}_score'] = 1
    else:
        # Others: higher is worse
        df.loc[col > THRESHOLDS[param]['high'], f'{param}_score'] = 2
        df.loc[(col > THRESHOLDS[param]['medium']) & (col <= THRESHOLDS[param]['high']), f'{param}_score'] = 1
    df.loc[col.isna(), f'{param}_score'] = np.nan

# Overall risk = max score (worst-case)
score_cols = [f'{p}_score' for p in THRESHOLDS.keys() if f'{p}_score' in df.columns]
df['overall_score'] = df[score_cols].max(axis=1, skipna=True)

# Map to Risk labels
df['Risk'] = df['overall_score'].map({0: 'Low', 1: 'Medium', 2: 'High', np.nan: np.nan})
df['Risk'] = df['Risk'].astype('category')

print(f"Risk distribution:\n{df['Risk'].value_counts(dropna=False)}")
print(f"\nRows with Risk: {df['Risk'].notna().sum()} / {len(df)}")



Risk distribution:
Risk
High      1203
Medium     714
Low         70
NaN          4
Name: count, dtype: int64

Rows with Risk: 1987 / 1991


In [ ]:
# Forecast model training with lagged features
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib
from pathlib import Path

# Config
H, L = 1, 3  # forecast horizon, lookback
numeric_features = ['Temp','DO','pH','Conductivity','BOD','Nitrate','FecalColiform','TotalColiform']
features = [f for f in numeric_features if f in df.columns]
MODEL_OUTPUT_FORECAST = Path(MODEL_OUTPUT).parent / 'rf_forecast_model.joblib'

# Auto-detect station column
station_col = None
if 'StationCode' in df.columns:
    station_col = 'StationCode'
elif 'MonitoringLocation' in df.columns:
    station_col = 'MonitoringLocation'

# Build lagged dataset
rows = []
if station_col:
    for station_id in df[station_col].dropna().unique():
        station_df = df[df[station_col] == station_id].reset_index(drop=True)
        for t in range(L-1, len(station_df) - H):
            if pd.notna(station_df.loc[t+H, 'Risk']):
                lag_row = {}
                for feat in features:
                    for lag in range(L):
                        lag_row[f'{feat}_lag{lag}'] = station_df.loc[t-lag, feat] if pd.notna(station_df.loc[t-lag, feat]) else np.nan
                lag_row['station_id'] = station_id
                lag_row['target'] = station_df.loc[t+H, 'Risk']
                rows.append(lag_row)
else:
    # Single series
    for t in range(L-1, len(df) - H):
        if pd.notna(df.loc[t+H, 'Risk']):
            lag_row = {}
            for feat in features:
                for lag in range(L):
                    lag_row[f'{feat}_lag{lag}'] = df.loc[t-lag, feat] if pd.notna(df.loc[t-lag, feat]) else np.nan
            lag_row['target'] = df.loc[t+H, 'Risk']
            rows.append(lag_row)

X_df = pd.DataFrame(rows)
if len(X_df) == 0:
    raise ValueError("No valid lagged rows created. Check data and Risk column.")

# Encode target
le_fore = LabelEncoder()
y = le_fore.fit_transform(X_df['target'].astype(str))

# Encode station if present
oe = None
if station_col and 'station_id' in X_df.columns:
    oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    X_df['station_encoded'] = oe.fit_transform(X_df[['station_id']])
    lag_features = [c for c in X_df.columns if c.endswith('_lag0') or c.endswith('_lag1') or c.endswith('_lag2')] + ['station_encoded']
else:
    lag_features = [c for c in X_df.columns if c.endswith('_lag0') or c.endswith('_lag1') or c.endswith('_lag2')]

X = X_df[lag_features].fillna(X_df[lag_features].median())

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train model
rf = RandomForestClassifier(n_estimators=200, max_depth=12, n_jobs=-1, random_state=42)
rf.fit(X_train, y_train)

# Evaluate
y_pred = rf.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred):.4f}')
print('\nClassification Report:')
print(classification_report(y_test, y_pred, target_names=le_fore.classes_))

# Save
joblib.dump({'model': rf, 'label_encoder': le_fore, 'lag_features': lag_features, 'L': L, 'H': H, 'station_col': station_col, 'station_encoder': oe}, MODEL_OUTPUT_FORECAST)
print(f'\nModel saved to {MODEL_OUTPUT_FORECAST}')



Accuracy: 0.8241

Classification Report:
              precision    recall  f1-score   support

        High       0.87      0.89      0.88       139
         Low       0.67      0.29      0.40         7
      Medium       0.73      0.74      0.74        70

    accuracy                           0.82       216
   macro avg       0.76      0.64      0.67       216
weighted avg       0.82      0.82      0.82       216


Model saved to rf_forecast_model.joblib


In [ ]:
# Live graph visualization from synthetic sensor data
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from datetime import datetime
import time
import os

# Path to sensor data file (created by synthetic_sensors.py)
SENSOR_DATA_FILE = Path("sensor_live_data.csv")

def create_live_graph(max_points=50, update_interval=1000):
    """
    Create a live updating graph from synthetic sensor data.
    
    Parameters:
    - max_points: Maximum number of data points to display (default: 50)
    - update_interval: Animation update interval in milliseconds (default: 1000ms)
    """
    # Initialize data storage
    timestamps = []
    readings_data = {
        'Temp': [],
        'DO': [],
        'pH': [],
        'Risk_Score': []  # Numeric risk: 0=Low, 1=Medium, 2=High
    }
    risk_labels = []
    
    # Risk to numeric mapping
    risk_to_num = {'Low': 0, 'Medium': 1, 'High': 2}
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Live Water Quality Monitoring - Synthetic Sensor Data', fontsize=16, fontweight='bold')
    
    # Subplot 1: Temperature
    ax1 = axes[0, 0]
    line1, = ax1.plot([], [], 'b-', linewidth=2, marker='o', markersize=4)
    ax1.set_title('Temperature (°C)', fontweight='bold')
    ax1.set_ylabel('Temperature (°C)')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(15, 40)
    
    # Subplot 2: Dissolved Oxygen
    ax2 = axes[0, 1]
    line2, = ax2.plot([], [], 'g-', linewidth=2, marker='s', markersize=4)
    ax2.set_title('Dissolved Oxygen (mg/L)', fontweight='bold')
    ax2.set_ylabel('DO (mg/L)')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 12)
    
    # Subplot 3: pH Level
    ax3 = axes[1, 0]
    line3, = ax3.plot([], [], 'r-', linewidth=2, marker='^', markersize=4)
    ax3.set_title('pH Level', fontweight='bold')
    ax3.set_ylabel('pH')
    ax3.set_xlabel('Time')
    ax3.grid(True, alpha=0.3)
    ax3.set_ylim(5, 10)
    ax3.axhspan(6.5, 8.5, alpha=0.2, color='green', label='Safe Range')
    
    # Subplot 4: Risk Level
    ax4 = axes[1, 1]
    line4, = ax4.plot([], [], 'purple', linewidth=2, marker='D', markersize=4)
    ax4.set_title('Predicted Risk Level', fontweight='bold')
    ax4.set_ylabel('Risk (0=Low, 1=Medium, 2=High)')
    ax4.set_xlabel('Time')
    ax4.grid(True, alpha=0.3)
    ax4.set_ylim(-0.5, 2.5)
    ax4.set_yticks([0, 1, 2])
    ax4.set_yticklabels(['Low', 'Medium', 'High'])
    ax4.axhspan(0, 0.5, alpha=0.2, color='green', label='Low Risk')
    ax4.axhspan(0.5, 1.5, alpha=0.2, color='yellow', label='Medium Risk')
    ax4.axhspan(1.5, 2.5, alpha=0.2, color='red', label='High Risk')
    
    plt.tight_layout()
    
    def read_sensor_data():
        """Read latest data from sensor CSV file"""
        if not SENSOR_DATA_FILE.exists():
            return None
        
        try:
            # Read the CSV file
            df = pd.read_csv(SENSOR_DATA_FILE)
            if len(df) == 0:
                return None
            
            # Get the latest row
            latest = df.iloc[-1]
            
            # Parse timestamp
            try:
                ts = pd.to_datetime(latest['timestamp'])
            except:
                ts = datetime.now()
            
            return {
                'timestamp': ts,
                'Temp': latest.get('Temp', np.nan),
                'DO': latest.get('DO', np.nan),
                'pH': latest.get('pH', np.nan),
                'Risk': latest.get('Risk', 'Unknown')
            }
        except Exception as e:
            print(f"Error reading sensor data: {e}")
            return None
    
    def update_graph(frame):
        """Animation update function"""
        # Read latest sensor data
        data = read_sensor_data()
        
        if data is None:
            return line1, line2, line3, line4
        
        # Add new data point
        timestamps.append(data['timestamp'])
        readings_data['Temp'].append(data['Temp'])
        readings_data['DO'].append(data['DO'])
        readings_data['pH'].append(data['pH'])
        
        # Convert risk to numeric
        risk_num = risk_to_num.get(data['Risk'], 1)
        readings_data['Risk_Score'].append(risk_num)
        risk_labels.append(data['Risk'])
        
        # Keep only last max_points
        if len(timestamps) > max_points:
            timestamps.pop(0)
            readings_data['Temp'].pop(0)
            readings_data['DO'].pop(0)
            readings_data['pH'].pop(0)
            readings_data['Risk_Score'].pop(0)
            risk_labels.pop(0)
        
        # Update plots
        if len(timestamps) > 0:
            # Convert timestamps to relative time (seconds from start)
            if len(timestamps) == 1:
                time_axis = [0]
            else:
                start_time = timestamps[0]
                time_axis = [(t - start_time).total_seconds() for t in timestamps]
            
            line1.set_data(time_axis, readings_data['Temp'])
            line2.set_data(time_axis, readings_data['DO'])
            line3.set_data(time_axis, readings_data['pH'])
            line4.set_data(time_axis, readings_data['Risk_Score'])
            
            # Update axis limits
            if len(time_axis) > 1:
                ax1.set_xlim(min(time_axis), max(time_axis))
                ax2.set_xlim(min(time_axis), max(time_axis))
                ax3.set_xlim(min(time_axis), max(time_axis))
                ax4.set_xlim(min(time_axis), max(time_axis))
            else:
                ax1.set_xlim(0, 10)
                ax2.set_xlim(0, 10)
                ax3.set_xlim(0, 10)
                ax4.set_xlim(0, 10)
        
        return line1, line2, line3, line4
    
    # Create animation
    ani = animation.FuncAnimation(fig, update_graph, interval=update_interval, blit=True, cache_frame_data=False)
    
    print("📊 Live graph started. Make sure synthetic_sensors.py is running with 'on' command.")
    print(f"   Reading from: {SENSOR_DATA_FILE.absolute()}")
    print("   Close the graph window to stop.")
    
    plt.show()
    return ani

# To start the live graph, run:
# ani = create_live_graph(max_points=50, update_interval=1000)

